In [ ]:
import os


In [1]:
! huggingface-cli download qwen/Qwen2.5-7B-Instruct --max-workers 16

Fetching 14 files:   0%|                                 | 0/14 [00:00<?, ?it/s]Downloading 'config.json' to '/root/.cache/huggingface/hub/models--qwen--Qwen2.5-7B-Instruct/blobs/0178295f88afc3c7f279ed284f961f8c1be00654.incomplete'

.gitattributes: 1.52kB [00:00, 3.97MB/s]                                        
Download complete. Moving file to /root/.cache/huggingface/hub/models--qwen--Qwen2.5-7B-Instruct/blobs/a6344aac8c09253b3b630fb776ae94478aa0275b
Fetching 14 files:   7%|█▊                       | 1/14 [00:00<00:12,  1.02it/s]
LICENSE: 11.3kB [00:00, 17.7MB/s]
Download complete. Moving file to /root/.cache/huggingface/hub/models--qwen--Qwen2.5-7B-Instruct/blobs/6634c8cc3133b3848ec74b9f275acaaa1ea618ab

generation_config.json: 100%|███████████████████| 243/243 [00:00<00:00, 950kB/s]
Download complete. Moving file to /root/.cache/huggingface/hub/models--qwen--Qwen2.5-7B-Instruct/blobs/0eb3c536657dcd12626e09eca4b6198c0cbcde1e

config.json: 663B [00:00, 1.88MB/s]                     

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from rouge import Rouge
import sys
sys.path.append("/workspace/packges")
from MyDatasets import loading_dataset, collote_fn
import torch
import json

In [ ]:
args = {
        "aux_name": "meta-llama/Llama-3.1-8B-Instruct",
        "base_name": "meta-llama/Llama-2-7b-hf",
        "dataset": "/datasets/mandarjoshi/trivia_qa/rc.nocontext/rc_nocontext_validation.json",
        "batch_size": 1,
        "shuffle": True,
        "seed": 42,
        "loss_func": "all",
        "mode": "validation",
        "device": "cpu",
        "use_prompt": True
    }

In [ ]:
aux_tokenizer = AutoTokenizer.from_pretrained(args["aux_name"])
aux = AutoModelForCausalLM.from_pretrained(args["aux_name"])
aux.eval()
aux_tokenizer.pad_token = aux_tokenizer.eos_token

In [ ]:
dataset = loading_dataset(args["dataset"],use_prompt=args["use_prompt"])
dataloader = DataLoader(dataset, batch_size=args["batch_size"], shuffle=args["shuffle"], collate_fn=collote_fn)

In [ ]:
dataset[0]

In [ ]:
rouge = Rouge()
for batched_data in tqdm(dataloader):
    inputs = aux_tokenizer(batched_data["question"], return_tensors="pt", padding=True, truncation=True)
    pred_ids = []
    # for i in range(3):
    #     outputs = aux(
    #         **inputs, 
    #         output_hidden_states=True, 
    #         output_attentions=True, 
    #         use_cache=True, 
    #         predicted_key_value=None, 
    #     )
        
    #     logits = outputs.logits[:, -1, :]
    #     pred_id = torch.argmax(logits, dim=-1)  # Greedy decoding
    #     if i == 0:
    #         pred_ids = pred_id.unsqueeze(1)
    #     else:
    #         pred_ids = torch.cat([pred_ids, pred_id.unsqueeze(1)], dim=-1)
        
    #     if all(pred_id == aux_tokenizer.eos_token_id):
    #         break
    #     # update the inputs
        
    #     inputs["input_ids"] = torch.cat([inputs["input_ids"], pred_id.unsqueeze(1)], dim=-1)
    #     inputs["attention_mask"] = torch.cat([inputs["attention_mask"], torch.ones_like(pred_id).unsqueeze(1)], dim=-1)
    #     print(inputs["input_ids"])
    #     print(inputs["attention_mask"])
    
    outputs = aux.generate(
        **inputs, 
        max_new_tokens=5,
        pad_token_id=aux_tokenizer.eos_token_id,
    )
    
    preds = aux_tokenizer.batch_decode(outputs[:,inputs["input_ids"].shape[1]:], skip_special_tokens=True)
    refs = aux_tokenizer(batched_data["answer"], return_tensors="pt", padding=True, truncation=True)
    refs = aux_tokenizer.batch_decode(refs["input_ids"], skip_special_tokens=True)
    print(json.dumps(preds,indent=4))
    print(json.dumps(refs,indent=4))
    temp_scores = rouge.get_scores(hyps=preds, refs=refs, avg=True)
    temp_results = {key: value['f'] for key, value in temp_scores.items()}
    temp_results['avg'] = sum(temp_results.values()) / len(temp_results)
    print(f"current {args['mode']} Rouge-1: {temp_results['rouge-1']:>0.2f} Rouge-2: {temp_results['rouge-2']:>0.2f} Rouge-L: {temp_results['rouge-l']:>0.2f} average: {temp_results['avg']}\n")

In [ ]:
import requests
import json

def get_completion(prompt):
    headers = {'Content-Type': 'application/json'}
    data = {"prompt": prompt}
    response = requests.post(url='http://127.0.0.1:6006', headers=headers, data=json.dumps(data))
    return response.json()['response']

if __name__ == '__main__':
    print(get_completion('你好'))